# Ejemplo de red recurrente LSTM

## Implementada con Keras y que genera texto, aprendiendo anivel de caracter teniendo como base de entrenamiento el libro "Don Quijote de la Mancha"

In [12]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
file = 'https://www.gutenberg.org/files/2000/2000-0.txt' # link a la versión txt UTF-8 del libro en la web del proyecto Gutemberg
got_file = tf.keras.utils.get_file('Don-Quijote.txt', file)

print(got_file) # imprime la ruta al fichero descargado

texto = open(got_file, 'rb').read().decode(encoding='utf-8')
print(f'Longitud del dataset: {len(texto)} caracteres.')

# al convertir a conjunto, se eliminan caracteres duplicados, así que en
# vocabulario se guarda un ejemplar de cada caracter de los que se encuentran
# en el texto
vocabulario = sorted(set(texto))
print(f'Tamaño del vocabulario: {len(vocabulario)} caracteres.')


2226045/2226045 [==============================] - 0s 0us/step
/root/.keras/datasets/Don-Quijote.txt
Longitud del dataset: 2168460 caracteres.
Tamaño del vocabulario: 106 caracteres.


In [24]:
# creamos los diccionarios que codifican cada caracter a número y cada número a caracter...
char_index = {u:i for i,u in enumerate(vocabulario)}
index_char = {i:u for i,u in enumerate(vocabulario)}
print(char_index)
print(index_char)

codigos = [char_index[c] for c in texto[:100]]
caracteres = [index_char[cod] for cod in codigos]
print(f'Primeros 100 caracteres del libro codificados:\n {codigos}')
print(f'Primeros 100 caracteres del libro DEcodificados:\n {caracteres}')

{'\n': 0, '\r': 1, ' ': 2, '!': 3, '"': 4, '#': 5, '$': 6, '%': 7, "'": 8, '(': 9, ')': 10, '*': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '?': 28, '@': 29, 'A': 30, 'B': 31, 'C': 32, 'D': 33, 'E': 34, 'F': 35, 'G': 36, 'H': 37, 'I': 38, 'J': 39, 'K': 40, 'L': 41, 'M': 42, 'N': 43, 'O': 44, 'P': 45, 'Q': 46, 'R': 47, 'S': 48, 'T': 49, 'U': 50, 'V': 51, 'W': 52, 'X': 53, 'Y': 54, 'Z': 55, '[': 56, ']': 57, 'a': 58, 'b': 59, 'c': 60, 'd': 61, 'e': 62, 'f': 63, 'g': 64, 'h': 65, 'i': 66, 'j': 67, 'k': 68, 'l': 69, 'm': 70, 'n': 71, 'o': 72, 'p': 73, 'q': 74, 'r': 75, 's': 76, 't': 77, 'u': 78, 'v': 79, 'w': 80, 'x': 81, 'y': 82, 'z': 83, '¡': 84, '«': 85, '»': 86, '¿': 87, 'Á': 88, 'É': 89, 'Í': 90, 'Ñ': 91, 'Ó': 92, 'Ú': 93, 'à': 94, 'á': 95, 'é': 96, 'í': 97, 'ï': 98, 'ñ': 99, 'ó': 100, 'ù': 101, 'ú': 102, 'ü': 103, '—': 104, '\ufeff': 105}
{0: '\n', 1: '\r', 2: ' ', 3: '!', 4: '"',

In [5]:
# Para entrenar el modelo se tiene que preparar el dataset en secuencias de caracteres
# es decir, no le podemos "enchufar" todo el libro del tirón, sino que, del mismo modo que
# se entrena una red neuronal con una secuencia de diferentes imágenes (entrenamiento supervisado)
# en este caso, a partir del libro, generamos secuencias de fragmentos del libro.
# además deberán ser secuencias de los caracteres ya codificados...

libro_codificado = [char_index[c] for c in texto]

seq_length = 100 # tamaño de las secuencias

# este es el dataset anterior pero adaptado al formato "Keras/TensorFlow"
cod_dataset = tf.data.Dataset.from_tensor_slices(libro_codificado)

secuencias = cod_dataset.batch(seq_length+1, drop_remainder=True)

# Se implementan las secuencias de seq_length+1 para que, se tome como entrada
# el trozo de (0..seq_length) y como salida de (1 .. seq_length+1)

print('Primeras 3 secuencias:\n')
for seq in secuencias.take(3):
  # seq.numpy() --> convierte tensor en array
  cad = ''.join([index_char[cod] for cod in seq.numpy()])
  print(f'{repr(cad)}\n')


Primeras 3 secuencias:

'\ufeffThe Project Gutenberg eBook of Don Quijote, by Miguel de Cervantes Saavedra\r\n\r\nThis eBook is for the'

' use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with a'

'lmost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the'



In [10]:
# Ahora, que ya sabemos que tenemos las secuencias y que podemos codificar y
# decodificar según nuestros intereses, generamos el dataset

def generar_fila_dastaset(fila):
    input = fila[:-1]
    output = fila[1:]
    return input, output

# El dataset se almacena en formato "Keras/Tensorflow" de modo que se procese
# correctamente por nuestra librería durante el entrenamiento, y consta de un
# conjunto de secuencias de texto, de modo que el output elimina un caracter de
# la izquierda del input y agrega un caracter nuevo a la derecha
dataset = secuencias.map(generar_fila_dastaset)
print(f'{dataset}\n')

# Veamos una muestra cualquiera de entrada y salida
for input, output in dataset.take(1):
    entrada = ''.join([index_char[cod] for cod in input.numpy()])
    salida = ''.join([index_char[cod] for cod in output.numpy()])
    print(f'Input: {repr(entrada)}\nOutput: {repr(salida)}')

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>

Input: '\ufeffThe Project Gutenberg eBook of Don Quijote, by Miguel de Cervantes Saavedra\r\n\r\nThis eBook is for th'
Output: 'The Project Gutenberg eBook of Don Quijote, by Miguel de Cervantes Saavedra\r\n\r\nThis eBook is for the'


In [14]:
# Ahora se preparan los hiperparámetros fundamentales para entrenar nuestra red

BATCH_SIZE = 64
BUFFER_SIZE = 10000

# barajamos el dataset y lo organizamos en batches de, en este caso, 64 muestras
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)


<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>


In [16]:
# Construcción del modelo

# Nuestro modelo consta de. únicamente, tres capas (en cualquiera de ellas, pero
# especialmente en la capa LSTM pùedes observar algunos parámetros con los que
# jugar, como el inicializador recurrente).Te recomiento que profundices, no
# solo con el libro de Chollet, sino también con la referencia de Keras para
# comprender los diferentes hiperparámetros de las capas LSTM y que hagas
# pruebas con ellos
def crear_modelo(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential()
  # La primera capa genera, a partir del vocabulario, un embedding, por lo que
  # mapeará, cada carácter de entrada en un vector
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      batch_input_shape=[batch_size, None]))

  # La segunda capa es LSTM, que le da el comportamiento recurrente a
  # nuestro modelo. Revisa los argumentos en la documentación de Keras,
  # modifica alguno de ellos y ve observando cambios en el resultado final
  model.add(LSTM(units=rnn_units, return_sequences=True, stateful=True,
                 recurrent_initializer='glorot_uniform'))

  # Finalmente incorporaremos una capa Densa. No vamos a detenernos a
  # explicar este concepto, ya que, además, es una de las capas más
  # sencillas. En la bibliografía fundamental (Chollet o Torres,
  # por ejemplo) encontrarás, si lo necesitas, información suficiente.
  # al no tener una función de activación, retornará un vector con un
  # indicador de probabilidad para cada carácter
  model.add(Dense(units=vocab_size))

  return model

vocab_size = len(vocabulario)
embedding_dim = 256
rnn_units = 1024


modelo = crear_modelo(vocab_size=vocab_size, embedding_dim=embedding_dim,
                      rnn_units=rnn_units, batch_size=BATCH_SIZE)

modelo.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           27136     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 106)           108650    
                                                                 
Total params: 5,382,762
Trainable params: 5,382,762
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Se puede observar cómo nuestro modelo devuelve un tensor de salida, con una
# dimensión adicional, que es la verosimilitud (no probabilidad, que se
# obtendría en todo caso si aplicáramos softmax como función de activación
# de la capa densa) para cada uno de los caracteres del vocabulario

for input, output in dataset.take(1):
  pred = modelo(input)
  print(f'Pred: {pred.shape} (batch_size, sequence_length, vocab_size)')

# Para oredecir el carácter siguiente, el lector podría pensar en emplear el
# carácter más probable, mediante argmax(), pero en lugar de ello, y para
# prevenir un posible error obtendremos una muestra probable a partir de la
# distribución de salida.

# se obtiene una muestra aleatoria de una distribución categórica
indices = tf.random.categorical(pred[0], num_samples=1)

# squeeze elimina la dimensión adicional de un vector de tamaño 1
cars = tf.squeeze(indices, axis=-1).numpy()
print(cars)

Pred: (64, 100, 106) (batch_size, sequence_length, vocab_size)
[ 15 104  83  73  71  85 102  35  79  51  91  76 103  16  78  38  92   7
  76   4  39  27  73   0  84  11  60  59  90  66  17  97  61  54  59   8
  40   3  44  86  67  62  75  17  14  81  62  20  82  12  49  69  22  78
  24  38  89  71  74  50  10  45  21  15  47  99  27  63  31  95  30  52
  47  67  18  87  87  78  12  32  37  44  29  39   4  11  95  98  31  19
  78  98  27  46  70  59  59  14   7  96]


In [19]:
# Compilación del modelo creado

# Claramente se trata de un modelo clasificador, que va a predecir el carácter
# que, más probablemente, sigue a partir de la entrada.

# Para entrenar el modelo debemos definir una función de pérdida -loss-
# y un optimizador.

def perdida(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,
                                                         logits,
                                                         from_logits=True)

modelo.compile(optimizer='adam', loss=perdida)

In [20]:
# Entrenamiento del modelo

# Debido a que es un entrenamiento largo, y para ilustrar eluso de los
# checkpoints (que pueden emplearse para realizar predicciones, por cierto),
# vamos a utilizar checkpoints con Keras durante este entrenamiento.

# Se recomienda al lector que se documente, también, al respecto.

dir_checkpnt = './train_LSTM_OBS_Checkpoints'
check_pnt_prefix = os.path.join(dir_checkpnt, 'chkpt_{epoch}')

chkpnt_cb = tf.keras.callbacks.ModelCheckpoint(filepath=check_pnt_prefix,
                                               save_weights_only=True)

EPOCHS=50
history = modelo.fit(dataset, epochs=EPOCHS, callbacks=[chkpnt_cb])


Epoch 1/50
335/335 [==============================] - 34s 84ms/step - loss: 2.1771
Epoch 2/50
335/335 [==============================] - 28s 77ms/step - loss: 1.5511
Epoch 3/50
335/335 [==============================] - 29s 77ms/step - loss: 1.3700
Epoch 4/50
335/335 [==============================] - 28s 77ms/step - loss: 1.2888
Epoch 5/50
335/335 [==============================] - 28s 77ms/step - loss: 1.2366
Epoch 6/50
335/335 [==============================] - 28s 77ms/step - loss: 1.1982
Epoch 7/50
335/335 [==============================] - 28s 76ms/step - loss: 1.1668
Epoch 8/50
335/335 [==============================] - 28s 78ms/step - loss: 1.1382
Epoch 9/50
335/335 [==============================] - 29s 78ms/step - loss: 1.1113
Epoch 10/50
335/335 [==============================] - 27s 76ms/step - loss: 1.0863
Epoch 11/50
335/335 [==============================] - 27s 76ms/step - loss: 1.0609
Epoch 12/50
335/335 [==============================] - 29s 80ms/step - loss: 1.0361
E

In [21]:
# Generación de texto con el modelo entrenado.

# Se van a restaurar los pesos desde el último checkpoint, y además, para
# simplificar la generación se va a mantener un batch_size de 1 en la predicción

modelo_ok = crear_modelo(vocab_size=vocab_size, embedding_dim=embedding_dim,
                         rnn_units=rnn_units, batch_size=1)
modelo_ok.load_weights(tf.train.latest_checkpoint(checkpoint_dir=dir_checkpnt))

# Debido a que el batch_size es diferente, reconstruimos el modelo
modelo_ok.build(tf.TensorShape((1, None)))


def generar_texto(modelo, cadena_inicio, temperatura):
  num_generate=500 # se generarán 500 caracteres
  input = [char_index[car] for car in cadena_inicio]
  input = tf.expand_dims(input, 0)
  texto = []

  # indicará cómo de conservador es el modelo en sus predicciones
  # se recomienda hacer pruebas con valores más elevados (menos conservador)
  # y más bajos (más conservador)
  temp = temperatura

  modelo.reset_states()
  for i in range(num_generate):
    preds = modelo(input)
    preds = tf.squeeze(preds, 0)
    preds = preds / temp
    pred_id = tf.random.categorical(preds, num_samples=1)[-1,0].numpy()
    input = tf.expand_dims([pred_id], 0)
    texto.append(index_char[pred_id])

  return texto

In [26]:
# Ejemplo de generación

print('Texto generado sin demasiada creatividad:')
print('-------------------------------------------------------------')
texto = generar_texto(modelo_ok, cadena_inicio='Rocinante', temperatura=0.3)
print(''.join(car for car in texto))
print()


print('Texto generado con una palabra que no está en el vocabulario:')
print('-------------------------------------------------------------')
texto = generar_texto(modelo_ok, cadena_inicio='red neuronal', temperatura=0.3)
print(''.join(car for car in texto))
print()


print('Dotemos al generador de mayor creatividad...:')
print('-------------------------------------------------------------')
texto = generar_texto(modelo_ok, cadena_inicio='escudero', temperatura=1)
print(''.join(car for car in texto))
print()


Texto generado sin demasiada creatividad:
-------------------------------------------------------------
, y como él se imaginaba que
había de hacer en el fragies y se lo dijo el mandamiento de su
deseo; que, puesto que los conocían que entre los dos se
dice que le he menester para condesa para el juez quién
ha sido la suya. En resolución, todos los de la carreta, donde le dejaremos por
la barca y voto a nosotros el vestido, por lo menos en más estraña catálantida. El cura los tambobernador mío, que no le quiero dejar de ser
tanto como el precio de la jineta un personaje, que todo lo demás pu

Texto generado con una palabra que no está en el vocabulario:
-------------------------------------------------------------
 de los Amadís de Gaula, que sería destruido de cabellos
andantes, que se lo avisé, con que se lo agradece, y que, sin
ser alcanzare la verdad de lo que más le fatigaba era de comer a la ciudad, al entrar de
su palafrén, acomodado y contento, y, puesto en pie, y, después de h